In [16]:
# insert_symbols.py
import datetime
from math import ceil
import bs4
import MySQLdb as mdb
import requests

In [1]:
import os
os.environ['ALGO_DIR']

'Users/chiemelienwanisobi/cloned_repos/TFP-Algo'

In [4]:

def obtain_parse_wiki_snp500():
    """
    Download and parse the Wikipedia list of S&P500
    constituents using requests and BeautifulSoup.
    Returns a list of tuples for to add to MySQL.
    """
    # Stores the current time, for the created_at record
    now = datetime.datetime.now(datetime.UTC)
    # Use requests and BeautifulSoup to download the
    # list of S&P500 companies and obtain the symbol table
    response = requests.get(
    "http://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    )
    soup = bs4.BeautifulSoup(response.text)
    # This selects the first table, using CSS Selector syntax
    # and then ignores the header row ([1:])
    symbolslist = soup.select('table')[0].select('tr')[1:]
    # Obtain the symbol information for each
    # row in the S&P500 constituent table
    symbols = []
    for i, symbol in enumerate(symbolslist):
        tds = symbol.select('td')
        symbols.append(
            (
            tds[0].select('a')[0].text, # Ticker
            'stock',
            tds[1].select('a')[0].text, # Name
            tds[3].text, # Sector
            'USD', now, now
            )
        )
    return symbols

sp = obtain_parse_wiki_snp500()
sp

[('MMM',
  'stock',
  '3M',
  'Industrial Conglomerates',
  'USD',
  datetime.datetime(2024, 7, 8, 2, 33, 7, 316986, tzinfo=datetime.timezone.utc),
  datetime.datetime(2024, 7, 8, 2, 33, 7, 316986, tzinfo=datetime.timezone.utc)),
 ('AOS',
  'stock',
  'A. O. Smith',
  'Building Products',
  'USD',
  datetime.datetime(2024, 7, 8, 2, 33, 7, 316986, tzinfo=datetime.timezone.utc),
  datetime.datetime(2024, 7, 8, 2, 33, 7, 316986, tzinfo=datetime.timezone.utc)),
 ('ABT',
  'stock',
  'Abbott',
  'Health Care Equipment',
  'USD',
  datetime.datetime(2024, 7, 8, 2, 33, 7, 316986, tzinfo=datetime.timezone.utc),
  datetime.datetime(2024, 7, 8, 2, 33, 7, 316986, tzinfo=datetime.timezone.utc)),
 ('ABBV',
  'stock',
  'AbbVie',
  'Biotechnology',
  'USD',
  datetime.datetime(2024, 7, 8, 2, 33, 7, 316986, tzinfo=datetime.timezone.utc),
  datetime.datetime(2024, 7, 8, 2, 33, 7, 316986, tzinfo=datetime.timezone.utc)),
 ('ACN',
  'stock',
  'Accenture',
  'IT Consulting & Other Services',
  'USD',
  d

In [23]:
def insert_snp500_symbols(symbols):
    """
    Insert the S&P500 symbols into the MySQL database.
    """
    # Connect to the MySQL instance
    db_host = 'localhost'
    db_user = 'home_user'
    db_pass = '787753'
    db_name = 'securities_master'
    con = mdb.connect(
    host=db_host, user=db_user, passwd=db_pass, db=db_name
    )
    # Create the insert strings
    column_str = (
    "ticker, instrument, name, sector, "
    "currency, created_date, last_updated_date"
    )
    insert_str = ("%s, " * 7)[:-2]
    final_str = "INSERT INTO symbol (%s) VALUES (%s)" % \
    (column_str, insert_str)
    # Using the MySQL connection, carry out
    # an INSERT INTO for every symbol
    cur = con.cursor()
    cur.executemany(final_str, symbols)
    con.commit()
    cur.close()
    con.close()
    # if __name__ == "__main__":
    #     symbols = obtain_parse_wiki_snp500()
    #     insert_snp500_symbols(symbols)
    #     print("%s symbols were successfully added." % len(symbols))


In [24]:
sp = obtain_parse_wiki_snp500()
insert_snp500_symbols(sp)

In [10]:
#Establish connection 
 # Connect to the MySQL instance
db_host = 'localhost'
db_user = 'home_user'
db_pass = '787753'
db_name = 'securities_master'
con = mdb.connect(
host=db_host, user=db_user, passwd=db_pass, db=db_name
)

cursor = con.cursor()

#id connections
cursor.execute("SELECT id from information_schema.processlist WHERE user = 'home_user'")
connections = cursor.fetchall()

#Terminate connections
for connection in connections: 
    kill_command = f"KILL {connection[0]}"
    cursor.execute(kill_command)

#Close cursor and connections 
cursor.close()
con.close()

OperationalError: (1317, 'Query execution was interrupted')

In [22]:
print(os.getenv('SQL_PASSWORD'))

None


In [1]:
%load_ext autoreload
%autoreload 2
from dbase.database.SQLHelpers import DatabaseAdapter
db = DatabaseAdapter()
db.query_database(
    'securities_master', 'temp_options_eod_new',
    'SELECT * FROM temp_options_eod_new WHERE underlier = "AAPL" LIMIT 200'

)


Console Logging & File Logging Can be configured using STREAM_LOG_LEVEL and FILE_LOG_LEVEL in environment variables.
Propagate to root logger can be set using PROPAGATE_TO_ROOT_LOGGER in environment variables.
Example:
STREAM_LOG_LEVEL = 'DEBUG'
FILE_LOG_LEVEL = 'INFO'
PROPAGATE_TO_ROOT_LOGGER = 'False'

2025-05-26 09:58:31 trade.helpers.Logging INFO: Logging Root Directory: /Users/chiemelienwanisobi/cloned_repos/QuantTools/logs
[get_engine] Creating engine for DB: securities_master, PID: 28777


,Open,High,Low,Close,Volume,Bid_size,CloseBid,Ask_size,CloseAsk,Strike,...,ask_binomial_theta,ask_binomial_vanna,ask_binomial_volga,bid_binomial_delta,bid_binomial_gamma,bid_binomial_vega,bid_binomial_rho,bid_binomial_theta,bid_binomial_vanna,bid_binomial_volga
0,0.00,0.00,0.00,0.00,0,51,122.20,73,125.55,310.0,...,0.001059,1.984156,277.444703,-0.994271,-1.421085e-10,0.000000,-3.228318,0.039365,4.328148e+07,3.686575e+13
1,0.00,0.00,0.00,0.00,0,57,121.10,56,123.45,310.0,...,0.005370,2.285318,356.611387,-0.994320,-2.842171e-10,0.000000,-3.220946,0.039333,4.276824e+07,3.617208e+13
2,0.00,0.00,0.00,0.00,0,51,119.35,104,122.20,310.0,...,0.000686,1.983819,277.954172,-0.994402,0.000000e+00,0.000000,-3.213217,0.039376,4.167311e+07,3.471681e+13
3,0.00,0.00,0.00,0.00,0,51,118.45,86,121.25,310.0,...,0.004123,2.212073,337.194796,-0.994420,2.842171e-10,0.000000,-3.205559,0.039405,4.167466e+07,3.469740e+13
4,0.00,0.00,0.00,0.00,0,58,118.55,84,121.10,310.0,...,0.004739,2.260673,350.197102,-0.994435,1.421085e-10,0.000000,-3.198108,0.039388,4.172780e+07,3.474740e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,29.30,30.09,28.54,28.60,81,1,27.15,1,29.00,170.0,...,-0.018751,0.539959,-20.899227,0.550210,5.227510e-03,0.940004,1.360941,-0.017645,5.498250e-01,-1.875084e+01
196,30.67,32.65,30.04,32.00,391,20,31.35,1,32.80,170.0,...,-0.019423,0.436778,-25.914265,0.583622,4.826836e-03,0.957839,1.458155,-0.018572,4.345077e-01,-2.459271e+01
197,32.15,34.90,32.15,34.65,3132,10,33.30,1,36.00,170.0,...,-0.020149,0.370171,-27.367822,0.601446,4.736985e-03,0.968010,1.530593,-0.018489,3.529095e-01,-2.456658e+01
198,34.65,35.60,34.00,34.30,340,150,33.90,46,35.10,170.0,...,-0.019803,0.376567,-26.790316,0.601125,4.608009e-03,0.964708,1.507586,-0.019063,3.697888e-01,-2.557206e+01


In [3]:
import signal
signal.getsignal(signal.SIGTERM)(1,1)


UnsupportedOperation: not writable

In [5]:
from dbase.database.SQLHelpers import _PROCESS_ENGINE_CACHE, _dispose_all_engines
_dispose_all_engines()